In [8]:
def calculate_padding(image):
    # 이미지의 높이와 너비를 가져옵니다.
    height, width = image.shape[:2]

    if width > height:
        # 가로가 세로보다 긴 경우:
        # 세로 길이를 가로 길이와 동일하게 만들기 위해 세로에 패딩을 추가합니다.
        padding_size = width - height  # 추가해야 할 세로 패딩 크기를 계산합니다.
        top_pad = padding_size // 2  # 패딩의 절반을 위쪽에 추가합니다.
        bottom_pad = padding_size - top_pad  # 나머지 패딩을 아래쪽에 추가합니다.
        left_pad, right_pad = 0, 0  # 가로 방향으로는 패딩이 필요하지 않으므로 0으로 설정합니다.
    elif height > width:
        # 세로가 가로보다 긴 경우:
        # 가로 길이를 세로 길이와 동일하게 만들기 위해 가로에 패딩을 추가합니다.
        padding_size = height - width  # 추가해야 할 가로 패딩 크기를 계산합니다.
        left_pad = padding_size // 2  # 패딩의 절반을 왼쪽에 추가합니다.
        right_pad = padding_size - left_pad  # 나머지 패딩을 오른쪽에 추가합니다.
        top_pad, bottom_pad = 0, 0  # 세로 방향으로는 패딩이 필요하지 않으므로 0으로 설정합니다.
    else:
        # 가로와 세로 길이가 동일한 경우:
        # 이미 정사각형이므로 패딩이 필요하지 않습니다.
        top_pad, bottom_pad, left_pad, right_pad = 0, 0, 0, 0

    # 각 방향의 패딩 크기를 반환합니다. (왼쪽, 아래쪽, 오른쪽, 위쪽 순서)
    return (left_pad, bottom_pad, right_pad, top_pad)


In [9]:
def normalize_with_padding(image, desired_size):
    # 원하는 출력 크기의 너비와 높이를 가져옵니다.
    desired_width, desired_height = desired_size

    # 입력 이미지의 원본 크기 가져오기
    height, width = image.shape[:2]

    # 원본 이미지와 원하는 크기의 가로/세로 비율 계산
    aspect_ratio = width / height  # 원본 이미지의 가로세로 비율
    desired_aspect_ratio = desired_width / desired_height  # 원하는 출력 크기의 가로세로 비율

    # 가로세로 비율을 유지하면서 이미지 크기를 조정
    if aspect_ratio > desired_aspect_ratio:
        # 원본 이미지가 가로로 더 긴 경우
        new_width = desired_width  # 새로운 너비는 원하는 크기의 너비에 맞춤
        new_height = int(new_width / aspect_ratio)  # 비율을 유지하여 새로운 높이를 계산
    else:
        # 원본 이미지가 세로로 더 긴 경우
        new_height = desired_height  # 새로운 높이는 원하는 크기의 높이에 맞춤
        new_width = int(new_height * aspect_ratio)  # 비율을 유지하여 새로운 너비를 계산

    # 크기가 조정된 이미지를 생성
    resized_image = cv2.resize(image, (new_width, new_height))

    # 이미지를 정규화하여 픽셀 값을 [0, 1] 범위로 조정
    resized_image = resized_image - np.min(resized_image)  # 최소값을 0으로 맞춤
    resized_image = resized_image / np.max(resized_image)  # 최대값을 1로 맞춤

    # 필요한 경우 패딩을 추가하여 원하는 크기로 맞춤
    top_pad = (desired_height - new_height) // 2  # 위쪽 패딩 크기 계산
    bottom_pad = desired_height - new_height - top_pad  # 아래쪽 패딩 크기 계산
    left_pad = (desired_width - new_width) // 2  # 왼쪽 패딩 크기 계산
    right_pad = desired_width - new_width - left_pad  # 오른쪽 패딩 크기 계산

    # 패딩을 추가하여 최종적으로 정사각형 형태의 이미지를 생성
    padded_image = cv2.copyMakeBorder(
        resized_image,
        top_pad,
        bottom_pad,
        left_pad,
        right_pad,
        cv2.BORDER_CONSTANT  # 패딩 영역은 고정된 값으로 채움
    )

    # 리사이즈 비율 계산 (너비 비율, 높이 비율)
    resize_ratio = (new_width / width, new_height / height)

    # 패딩 추가 여부에 따라 유효한 리사이즈 비율 결정
    if top_pad == 0 and bottom_pad == 0:
        # 세로 방향으로 패딩이 추가되지 않은 경우
        effective_resize_ratio = resize_ratio[1]  # 세로 비율 반환
    else:
        # 가로 방향으로 패딩이 추가되지 않은 경우
        effective_resize_ratio = resize_ratio[0]  # 가로 비율 반환

    # 패딩이 추가된 이미지와 유효한 리사이즈 비율 반환
    return padded_image, effective_resize_ratio


In [10]:
def IP_OBD_a(serial, dicom_path):
    # DICOM 파일 읽기
    dcm_a = pydicom.dcmread(dicom_path)  # 지정된 경로에서 DICOM 파일을 읽어옵니다.
    dcm_a_p = dcm_a.pixel_array.astype(np.float32)  # DICOM 데이터를 float32 형식으로 변환합니다.

    # 이미지가 세로로 길쭉한 경우(높이가 너비보다 100 이상 큰 경우) 이미지 회전
    if dcm_a_p.shape[0] > dcm_a_p.shape[1] + 100:
        dcm_a_p = cv2.rotate(dcm_a_p, cv2.ROTATE_90_COUNTERCLOCKWISE)  # 반시계 방향으로 90도 회전

    # 특정 serial 번호에 대해 이미지 추가 회전 수행
    if serial in [
        7, 10, 97, 127, 133, 181, 213, 216, 233, 268, 342, 366, 431, 444, 484, 487, 495, 499, 533, 534,
        574, 624, 647, 662, 683, 688, 692, 700, 704, 720, 754, 774, 760, 761, 786, 795, 806, 869, 888,
        904, 913, 927, 945, 961, 983, 992, 997, 1003, 1039, 1112, 1122, 1124, 1134, 1149, 1151, 1174,
        1226, 1240, 1259, 1260, 1265, 1278, 1300, 1310, 1329, 1339, 1343, 1356, 1414, 1429, 1436, 1451,
        1515, 1581, 1594, 1631, 1650, 1682, 1715, 1724, 1768, 1790, 1827, 1833, 1894, 1909, 1923, 1927,
        1958, 2015, 2017, 2040, 2041, 2044, 2045
    ]:
        dcm_a_p = cv2.rotate(dcm_a_p, cv2.ROTATE_180)  # 이미지를 180도 회전
    elif serial in [51, 182, 243, 259, 443, 598, 628, 804, 1276, 1543, 1562, 2012]:
        dcm_a_p = cv2.rotate(dcm_a_p, cv2.ROTATE_90_CLOCKWISE)  # 이미지를 시계 방향으로 90도 회전

    # RescaleIntercept 속성이 존재하는 경우 LUT(Look-Up Table)를 적용
    if hasattr(dcm_a, 'RescaleIntercept'):
        if dcm_a.RescaleIntercept >= 0:  # RescaleIntercept 값이 0 이상인 경우
            dcm_a_p = apply_modality_lut(dcm_a_p, dcm_a)  # LUT 변환 수행

    # WindowCenter 및 WindowWidth 속성이 있는 경우, 윈도우 레벨 조정
    if hasattr(dcm_a, 'WindowCenter') and hasattr(dcm_a, 'WindowWidth'):
        window_center = dcm_a.WindowCenter  # 윈도우 센터 값
        window_width = dcm_a.WindowWidth  # 윈도우 폭 값
        dcm_a_p = np.clip(
            dcm_a_p,
            window_center - (window_width / 2),  # 최소값
            window_center + (window_width / 2)   # 최대값
        )

    # 원본 DICOM 이미지를 저장
    Dicom_image_AP = dcm_a_p

    # 패딩 추가 및 크기 정규화 (목표 크기: 800x800)
    dcm_a_p, dcm_a_p_ratio = normalize_with_padding(dcm_a_p, (800, 800))  # 정규화된 이미지와 리사이즈 비율 계산
    dcm_a_p_pad = calculate_padding(Dicom_image_AP)  # 패딩 크기 계산
    dcm_a_p = dcm_a_p.astype(np.float32)  # 이미지를 float32 형식으로 변환

    # 원본 이미지, 정규화된 이미지, 리사이즈 비율, 패딩 크기 반환
    return Dicom_image_AP, dcm_a_p, dcm_a_p_ratio, dcm_a_p_pad


In [11]:
def IP_OBD_t(serial, dicom_path):
    # DICOM 파일 읽기
    dcm_t = pydicom.dcmread(dicom_path)  # 지정된 경로에서 DICOM 파일을 읽어옵니다.
    dcm_t_p = dcm_t.pixel_array.astype(np.float32)  # DICOM 데이터를 float32 형식으로 변환

    # 이미지가 세로로 길쭉한 경우(높이가 너비보다 100 이상 큰 경우) 이미지 회전
    if dcm_t_p.shape[0] > dcm_t_p.shape[1] + 100:
        dcm_t_p = cv2.rotate(dcm_t_p, cv2.ROTATE_90_COUNTERCLOCKWISE)  # 반시계 방향으로 90도 회전

    # 특정 serial 번호에 따라 이미지 회전 수행
    if serial in [
        10, 13, 16, 24, 40, 43, 45, 51, 56, 59, 61, 70, 76, 77, 83, 96, 106, 108, 123, 126, 127, 144,
        149, 160, 162, 164, 167, 169, 173, 183, 187, 188, 191, 192, 231, 236, 239, 241, 245, 250, 252,
        254, 256, 263, 267, 270, 272, 277, 283, 297, 310, 314, 317, 323, 334, 342, 354, 364, 366, 367,
        374, 378, 386, 388, 389, 394, 403, 414, 416, 419, 424, 432, 438, 441, 442, 451, 452, 461, 466,
        467, 472, 473, 483, 485, 494, 495, 496, 505, 514, 525, 539, 547, 548, 553, 554, 559, 575, 587,
        600, 608, 614, 622, 624, 629, 630, 632, 634, 641, 643, 644, 652, 658, 662, 669, 673, 674, 685,
        692, 699, 701, 706, 714, 717, 720, 731, 738, 742, 744, 755, 766, 768, 772, 774, 775, 776, 781,
        786, 793, 805, 813, 816, 817, 820, 822, 825, 826, 847, 850, 855, 858, 862, 902, 903, 911, 912,
        917, 919, 920, 927, 934, 941, 942, 945, 949, 952, 963, 965, 968, 970, 983, 985, 992, 997, 1002,
        1003, 1006, 1017, 1018, 1021, 1027, 1036, 1037, 1039, 1048, 1051, 1060, 1068, 1083, 1087, 1104,
        1115, 1119, 1132, 1136, 1143, 1149, 1151, 1153, 1161, 1175, 1181, 1183, 1184, 1188, 1189, 1198,
        1199, 1209, 1211, 1213, 1226, 1231, 1237, 1238, 1240, 1243, 1247, 1258, 1260, 1261, 1269, 1274,
        1276, 1277, 1281, 1290, 1291, 1292, 1303, 1310, 1324, 1329, 1332, 1339, 1342, 1343, 1346, 1350,
        1352, 1359, 1362, 1366, 1367, 1370, 1376, 1386, 1387, 1395, 1398, 1403, 1404, 1408, 1411, 1423,
        1428, 1436, 1438, 1439, 1445, 1449, 1451, 1456, 1461, 1468, 1472, 1474, 1479, 1485, 1488, 1508,
        1511, 1515, 1526, 1542, 1544, 1551, 1555, 1558, 1566, 1577, 1581, 1582, 1590, 1594, 1602, 1604,
        1607, 1610, 1617, 1636, 1640, 1643, 1647, 1652, 1653, 1656, 1660, 1665, 1670, 1678, 1681, 1686,
        1688, 1690, 1693, 1694, 1707, 1711, 1717, 1720, 1721, 1728, 1733, 1752, 1753, 1763, 1764, 1771,
        1776, 1779, 1786, 1792, 1808, 1816, 1825, 1842, 1855, 1856, 1860, 1869, 1871, 1873, 1877, 1880,
        1891, 1893, 1894, 1901, 1905, 1913, 1917, 1929, 1931, 1932, 1943, 1950, 1956, 1958, 1970, 1993,
        2032, 2036, 2040, 2044, 2126, 2131, 374, 1516, 1517
    ]:
        dcm_t_p = cv2.rotate(dcm_t_p, cv2.ROTATE_180)  # 180도 회전
    elif serial in [585, 1334]:
        dcm_t_p = cv2.rotate(dcm_t_p, cv2.ROTATE_90_COUNTERCLOCKWISE)  # 반시계 방향으로 90도 회전
    elif serial in [120, 642, 780, 996, 2028, 2101, 2252]:
        dcm_t_p = cv2.rotate(dcm_t_p, cv2.ROTATE_90_CLOCKWISE)  # 시계 방향으로 90도 회전
    elif serial in [2269]:
        dcm_t_p = rotate(dcm_t_p, angle=-30, reshape=False)  # -30도 회전

    # RescaleIntercept 속성이 있는 경우 LUT(Look-Up Table) 적용
    if hasattr(dcm_t, 'RescaleIntercept'):
        if dcm_t.RescaleIntercept >= 0:  # RescaleIntercept 값이 0 이상이면
            dcm_t_p = apply_modality_lut(dcm_t_p, dcm_t)

    # WindowCenter 및 WindowWidth 속성이 있는 경우 윈도우 레벨 조정
    if hasattr(dcm_t, 'WindowCenter') and hasattr(dcm_t, 'WindowWidth'):
        window_center = dcm_t.WindowCenter  # 윈도우 센터 값
        window_width = dcm_t.WindowWidth  # 윈도우 폭 값
        dcm_t_p = np.clip(
            dcm_t_p,
            window_center - (window_width / 2),  # 최소값
            window_center + (window_width / 2)   # 최대값
        )

    # 원본 DICOM 이미지를 저장
    Dicom_image = dcm_t_p

    # 패딩을 추가하여 이미지 정규화 (크기: 800x800)
    dcm_t_p, dcm_t_p_ratio = normalize_with_padding(dcm_t_p, (800, 800))  # 정규화된 이미지와 리사이즈 비율 반환
    dcm_t_p_pad = calculate_padding(Dicom_image)  # 패딩 크기 계산
    dcm_t_p = dcm_t_p.astype(np.float32)  # float32 형식으로 변환

    # 원본 이미지, 정규화된 이미지, 리사이즈 비율, 패딩 크기 반환
    return Dicom_image, dcm_t_p, dcm_t_p_ratio, dcm_t_p_pad


In [12]:
import xml.etree.ElementTree as ET
import torch

def Crop_image_a(Dicom_image_AP, xml_path, classes):
    # XML 파일 파싱
    tree = ET.parse(xml_path)  # 주어진 XML 파일을 파싱하여 트리 구조 생성
    root = tree.getroot()  # XML의 루트 노드 가져오기
    
    cropped_image_Left = None  # 왼쪽 영역의 크롭된 이미지 초기화
    cropped_image_Right = None  # 오른쪽 영역의 크롭된 이미지 초기화
    
    for obj in root.findall('object'):  # XML에서 모든 'object' 태그를 찾음
        class_name = obj.find('name').text  # 클래스 이름 가져오기
        if class_name not in classes:  # 지정된 클래스 목록에 없는 경우 건너뛰기
            continue

        label = classes.index(class_name)  # 클래스 이름을 인덱스로 변환

        # 바운딩 박스 좌표 가져오기
        box = obj.find('bndbox')
        xmin = float(box.find('xmin').text)  # 박스의 왼쪽 x좌표
        ymin = float(box.find('ymin').text)  # 박스의 위쪽 y좌표
        xmax = float(box.find('xmax').text)  # 박스의 오른쪽 x좌표
        ymax = float(box.find('ymax').text)  # 박스의 아래쪽 y좌표
        
        if class_name == 'Left':  # 'Left' 클래스에 해당하는 경우
            xmin_crop = int(xmin)  # 크롭 영역의 왼쪽 x좌표
            ymin_crop = int(ymin)  # 크롭 영역의 위쪽 y좌표
            xmax_crop = int(xmax)  # 크롭 영역의 오른쪽 x좌표
            ymax_crop = int(ymax)  # 크롭 영역의 아래쪽 y좌표
            cropped_image_Left = Dicom_image_AP[ymin_crop:ymax_crop, xmin_crop:xmax_crop]  # 이미지 크롭
            cropped_image_Left = cv2.resize(cropped_image_Left, (256, 256))  # 크롭된 이미지를 256x256으로 리사이즈
            cropped_image_Left = cropped_image_Left - np.min(cropped_image_Left)  # 최소값을 0으로 정규화
            cropped_image_Left = cropped_image_Left / np.max(cropped_image_Left)  # 최대값을 1로 정규화
            cropped_image_Left = cropped_image_Left.astype(np.float32)  # float32 형식으로 변환
            
        if class_name == 'Right':  # 'Right' 클래스에 해당하는 경우
            xmin_crop = int(xmin)  # 크롭 영역의 왼쪽 x좌표
            ymin_crop = int(ymin)  # 크롭 영역의 위쪽 y좌표
            xmax_crop = int(xmax)  # 크롭 영역의 오른쪽 x좌표
            ymax_crop = int(ymax)  # 크롭 영역의 아래쪽 y좌표
            cropped_image_Right = Dicom_image_AP[ymin_crop:ymax_crop, xmin_crop:xmax_crop]  # 이미지 크롭
            cropped_image_Right = cv2.resize(cropped_image_Right, (256, 256))  # 크롭된 이미지를 256x256으로 리사이즈
            cropped_image_Right = cropped_image_Right - np.min(cropped_image_Right)  # 최소값을 0으로 정규화
            cropped_image_Right = cropped_image_Right / np.max(cropped_image_Right)  # 최대값을 1로 정규화
            cropped_image_Right = cropped_image_Right.astype(np.float32)  # float32 형식으로 변환
        
    # 크롭된 왼쪽 및 오른쪽 이미지를 반환
    return cropped_image_Left, cropped_image_Right


In [13]:
import xml.etree.ElementTree as ET
import torch

def Crop_image_t(Dicom_image_LAT, xml_path, classes):
    # XML 파일 파싱
    tree = ET.parse(xml_path)  # 주어진 XML 파일을 파싱하여 트리 구조 생성
    root = tree.getroot()  # XML의 루트 노드 가져오기

    classes = ['LAT_Neck']  # LAT_Neck 클래스만 처리
    LAT_cropped_image = None  # 크롭된 LAT 이미지 초기화

    for obj in root.findall('object'):  # XML에서 모든 'object' 태그를 찾음
        class_name = obj.find('name').text  # 클래스 이름 가져오기
        if class_name not in classes:  # 클래스 이름이 지정된 목록에 없는 경우 건너뛰기
            continue

        label = classes.index(class_name)  # 클래스 이름을 인덱스로 변환

        # 바운딩 박스 좌표 가져오기
        box = obj.find('bndbox')
        xmin = float(box.find('xmin').text)  # 박스의 왼쪽 x좌표
        ymin = float(box.find('ymin').text)  # 박스의 위쪽 y좌표
        xmax = float(box.find('xmax').text)  # 박스의 오른쪽 x좌표
        ymax = float(box.find('ymax').text)  # 박스의 아래쪽 y좌표

        # 크롭 영역 설정
        xmin_crop = int(xmin)  # 크롭된 이미지의 왼쪽 x좌표
        ymin_crop = int(ymin)  # 크롭된 이미지의 위쪽 y좌표
        xmax_crop = int(xmax)  # 크롭된 이미지의 오른쪽 x좌표
        ymax_crop = int(ymax)  # 크롭된 이미지의 아래쪽 y좌표

        # DICOM 이미지를 크롭
        LAT_cropped_image = Dicom_image_LAT[ymin_crop:ymax_crop, xmin_crop:xmax_crop]
        LAT_cropped_image = cv2.resize(LAT_cropped_image, (256, 256))  # 크롭된 이미지를 256x256으로 리사이즈
        LAT_cropped_image = LAT_cropped_image - np.min(LAT_cropped_image)  # 최소값을 0으로 정규화
        LAT_cropped_image = LAT_cropped_image / np.max(LAT_cropped_image)  # 최대값을 1로 정규화
        LAT_cropped_image = LAT_cropped_image.astype(np.float32)  # float32 형식으로 변환

    # 크롭된 LAT 이미지를 반환
    return LAT_cropped_image


In [17]:
!pwd
import pandas as pd
import pydicom
import numpy as np
from sklearn.model_selection import train_test_split
import os
import pickle
from tqdm import tqdm
from pydicom.pixel_data_handlers.util import apply_modality_lut
from scipy.ndimage import rotate
import cv2

# Excel 파일 읽기 및 전처리
FNF_Excel = pd.read_excel('/workspace/FNF/labeling_detection_2to0.xlsx', usecols=[0, 7, 9, 11], engine='openpyxl', header=None)
FNF_Excel.columns = FNF_Excel.iloc[0]  # 첫 번째 행을 열 이름으로 설정
# 제외 기준에 따라 데이터 필터링
df_selected = FNF_Excel[['serial No.', 'exclusion', '정답label(by CT)']]
df_filtered = df_selected[df_selected['exclusion'] == 0]  # exclusion이 0인 데이터만 선택

# 통계 출력을 위한 초기 분석
print("\n=== 초기 데이터 분석 ===")
print("\nExclusion 값 분포:")
print(df_selected['exclusion'].value_counts())
print("\n선택된 데이터 (exclusion 0, 2):")
print("Exclusion 값별 개수:")
print(df_filtered['exclusion'].value_counts())
print("\nGarden Type 분포 (exclusion 0):")
print(df_filtered[df_filtered['exclusion'] == 0]['정답label(by CT)'].value_counts().sort_index())
print("\nGarden Type 분포 (exclusion 2):")
print(df_filtered[df_filtered['exclusion'] == 2]['정답label(by CT)'].value_counts().sort_index())

# 클래스 레이블 정의
classes_a = ['Left', 'Right']
classes_t = ['LAT_Neck']

# 데이터를 저장할 리스트 초기화
Serial_list, Garden_Type_list = [], []
Dicom_image_path_AP_list, Dicom_image_path_LAT_list = [], []
Detection_image_AP_list, Detection_image_LAT_list = [], []
Crop_AP_Right_image_list, Crop_AP_Left_image_list, Crop_LAT_image_list = [], [], []
Ratio_AP_list, Ratio_LAT_list = [], []
Pad_AP_list, Pad_LAT_list = [], []
Xml_path_AP_list, Xml_path_LAT_list = [], []

# tqdm으로 데이터 처리 진행 상황 표시
print("Processing data...")
for idx, row in tqdm(df_filtered.iterrows(), total=len(df_filtered), desc="Processing rows"):
    a_value = row['serial No.']  # serial No.
    p_value = row['정답label(by CT)']  # 정답 label

    # 파일 경로 생성
    folder_path = f'/workspace/FNF/nas/FNF/final_data/datasets/raw_internal/{a_value}'
    file_path_a = f'{folder_path}/{a_value}a000.dcm'
    file_path_t = f'{folder_path}/{a_value}t000.dcm'
    Xml_path_AP = f'/workspace/FNF/nas/FNF/final_data/Detect_Label/AP+Neck_label+100/{a_value}a.xml'
    Xml_path_LAT = f'/workspace/FNF/nas/FNF/final_data/Detect_Label/LAT_label+100/{a_value}t.xml'
    try:
        # AP 및 LAT 이미지 처리
        Dicom_image_AP, Detection_image_AP, AP_ratio, AP_pad = IP_OBD_a(a_value, file_path_a)
        Dicom_image_LAT, Detection_image_LAT, LAT_ratio, LAT_pad = IP_OBD_t(a_value, file_path_t)
        Crop_AP_Left_image, Crop_AP_Right_image = Crop_image_a(Dicom_image_AP, Xml_path_AP, classes_a)
        Crop_LAT_image = Crop_image_t(Dicom_image_LAT, Xml_path_LAT, classes_t)

        # 처리된 데이터를 리스트에 저장
        Serial_list.append(a_value)
        Dicom_image_path_AP_list.append(file_path_a)
        Dicom_image_path_LAT_list.append(file_path_t)
        Detection_image_AP_list.append(Detection_image_AP)
        Detection_image_LAT_list.append(Detection_image_LAT)
        Crop_AP_Right_image_list.append(Crop_AP_Right_image)
        Crop_AP_Left_image_list.append(Crop_AP_Left_image)
        Crop_LAT_image_list.append(Crop_LAT_image)
        Ratio_AP_list.append(AP_ratio)
        Ratio_LAT_list.append(LAT_ratio)
        Pad_AP_list.append(AP_pad)
        Pad_LAT_list.append(LAT_pad)
        Xml_path_AP_list.append(Xml_path_AP)
        Xml_path_LAT_list.append(Xml_path_LAT)
        Garden_Type_list.append(p_value - 1)  # Garden Type 값을 조정하여 저장

    except Exception as e:
        # 예외 발생 시 오류 메시지 출력 및 건너뛰기
        print(f"Error processing serial {a_value}: {e}")
        continue

# 클래스별 인덱스를 5개의 폴드로 나누기
print("Splitting data into folds...")
class_indices = {label: [i for i, l in enumerate(Garden_Type_list) if l == label] for label in range(4)}
folds = {f'fold{i+1}_indices': [] for i in range(5)}

for label, indices in class_indices.items():
    f1_indices, f2345_indices = train_test_split(indices, test_size=0.8, random_state=42)
    f2_indices, f345_indices = train_test_split(f2345_indices, test_size=0.75, random_state=42)
    f3_indices, f45_indices = train_test_split(f345_indices, test_size=0.66666, random_state=42)
    f4_indices, f5_indices = train_test_split(f45_indices, test_size=0.5, random_state=42)
    
    folds['fold1_indices'].extend(f1_indices)
    folds['fold2_indices'].extend(f2_indices)
    folds['fold3_indices'].extend(f3_indices)
    folds['fold4_indices'].extend(f4_indices)
    folds['fold5_indices'].extend(f5_indices)

# 프로세스별 데이터 딕셔너리 생성
processes = {
    'Paper': {
        'Serial': Serial_list,
        'Dicom_image_path_AP': Dicom_image_path_AP_list,
        'Dicom_image_path_LAT': Dicom_image_path_LAT_list,
        'Garden_Type': Garden_Type_list,
    },
    'AP_Detection': {
        'Serial': Serial_list,
        'Detection_image_AP': Detection_image_AP_list,
        'Ratio_AP_list': Ratio_AP_list,
        'Pad_AP_list': Pad_AP_list,
        'Xml_path_AP': Xml_path_AP_list,
    },
    'LAT_Detection': {
        'Serial': Serial_list,
        'Detection_image_LAT': Detection_image_LAT_list,
        'Ratio_LAT_list': Ratio_LAT_list,
        'Pad_LAT_list': Pad_LAT_list,
        'Xml_path_LAT': Xml_path_LAT_list,
    },
    'Classification': {
        'Serial': Serial_list,
        'Crop_AP_Right_image': Crop_AP_Right_image_list,
        'Crop_AP_Left_image': Crop_AP_Left_image_list,
        'Crop_LAT_image': Crop_LAT_image_list,
        'Garden_Type': Garden_Type_list,
    },
}

# 처리된 데이터를 각 프로세스별로 pickle 파일로 저장
pickle_save_dir = '/workspace/FNF/preprocessed_data/datasets/pickle_data_0218'
os.makedirs(pickle_save_dir, exist_ok=True)

print("Saving data...")
for process_name, process_data in tqdm(processes.items(), desc="Saving processes"):
    process_pickle_data = {f'fold{i+1}': {key: [process_data[key][j] for j in folds[f'fold{i+1}_indices']] 
                           for key in process_data.keys()} for i in range(5)}

    pickle_file_path = os.path.join(pickle_save_dir, f'FNF_{process_name}_data.pkl')
    with open(pickle_file_path, 'wb') as f:
        pickle.dump(process_pickle_data, f)
    
    print(f"{process_name} data saved to {pickle_file_path}")



def print_statistics(processes, folds):
    print("\n=== 데이터 통계 분석 ===")
    
    # 전체 데이터 통계
    total_samples = len(processes['Paper']['Serial'])
    print(f"\n전체 데이터 수: {total_samples}")
    
    garden_types = processes['Paper']['Garden_Type']
    unique_types = sorted(set(garden_types))
    
    # Exclusion 값별 통계
    excel_data = pd.read_excel('/workspace/FNF/labeling.xlsx')
    process_serials = set(processes['Paper']['Serial'])
    
    print("\n포함된 데이터의 Exclusion 분포:")
    for serial in process_serials:
        excel_row = excel_data[excel_data['serial No.'] == serial]
        if not excel_row.empty:
            exclusion_val = excel_row['exclusion'].iloc[0]
            garden_type = excel_row['정답label(by CT)'].iloc[0]
            print(f"Serial: {serial}, Exclusion: {exclusion_val}, Garden Type: {garden_type}")
    
    print("\n전체 Garden Type 분포:")
    for type_val in unique_types:
        count = garden_types.count(type_val)
        percentage = (count / total_samples) * 100
        print(f"Type {type_val + 1}: {count} ({percentage:.1f}%)")

    # Fold별 통계
    print("\nFold별 Garden Type 분포:")
    for fold_name, fold_indices in folds.items():
        fold_types = [garden_types[i] for i in fold_indices]
        print(f"\n{fold_name}:")
        print(f"총 데이터 수: {len(fold_indices)}")
        for type_val in unique_types:
            count = fold_types.count(type_val)
            percentage = (count / len(fold_indices)) * 100
            print(f"Type {type_val + 1}: {count} ({percentage:.1f}%)")

    # Serial number 범위
    serials = processes['Paper']['Serial']
    print(f"\nSerial Number 범위: {min(serials)} ~ {max(serials)}")
# 데이터 저장 전에 통계 출력
print_statistics(processes, folds)

# 처리된 데이터를 각 프로세스별로 pickle 파일로 저장
pickle_save_dir = '/workspace/FNF/preprocessed_data/datasets/pickle_data_1733'
os.makedirs(pickle_save_dir, exist_ok=True)

print("\nSaving data...")
for process_name, process_data in tqdm(processes.items(), desc="Saving processes"):
    process_pickle_data = {f'fold{i+1}': {key: [process_data[key][j] for j in folds[f'fold{i+1}_indices']] 
                           for key in process_data.keys()} for i in range(5)}

    pickle_file_path = os.path.join(pickle_save_dir, f'FNF_{process_name}_data.pkl')
    with open(pickle_file_path, 'wb') as f:
        pickle.dump(process_pickle_data, f)
    
    # 각 프로세스별 저장된 데이터 통계
    print(f"\n{process_name} 데이터 저장 완료:")
    print(f"파일 경로: {pickle_file_path}")
    print(f"Fold 수: {len(process_pickle_data)}")
    for fold_name, fold_data in process_pickle_data.items():
        print(f"{fold_name} 데이터 수: {len(fold_data['Serial'])}")

/workspace/FNF

=== 초기 데이터 분석 ===

Exclusion 값 분포:
0            1589
1             779
exclusion       1
Name: exclusion, dtype: int64

선택된 데이터 (exclusion 0, 2):
Exclusion 값별 개수:
0    1589
Name: exclusion, dtype: int64

Garden Type 분포 (exclusion 0):
1    379
2     69
3    476
4    664
Name: 정답label(by CT), dtype: int64

Garden Type 분포 (exclusion 2):
Series([], Name: 정답label(by CT), dtype: int64)
Processing data...


Processing rows: 100%|█████████████████████████████████████████████████████████████████████████████████████████| 1589/1589 [1:36:37<00:00,  3.65s/it]


Splitting data into folds...
Saving data...


Saving processes:   0%|                                                                                     | 0/4 [00:00<?, ?it/s]

Paper data saved to /workspace/FNF/preprocessed_data/datasets/pickle_data_0218/FNF_Paper_data.pkl


...


AP_Detection 데이터 저장 완료:
파일 경로: /workspace/FNF/preprocessed_data/datasets/pickle_data_1733/FNF_AP_Detection_data.pkl
Fold 수: 5
fold1 데이터 수: 315
fold2 데이터 수: 318
fold3 데이터 수: 318
fold4 데이터 수: 318
fold5 데이터 수: 319


Saving processes:  75%|█████████████████████████████████████████████████████████▊                   | 3/4 [01:09<00:24, 24.74s/it]


LAT_Detection 데이터 저장 완료:
파일 경로: /workspace/FNF/preprocessed_data/datasets/pickle_data_1733/FNF_LAT_Detection_data.pkl
Fold 수: 5
fold1 데이터 수: 315
fold2 데이터 수: 318
fold3 데이터 수: 318
fold4 데이터 수: 318
fold5 데이터 수: 319


Saving processes: 100%|█████████████████████████████████████████████████████████████████████████████| 4/4 [01:22<00:00, 20.58s/it]


Classification 데이터 저장 완료:
파일 경로: /workspace/FNF/preprocessed_data/datasets/pickle_data_1733/FNF_Classification_data.pkl
Fold 수: 5
fold1 데이터 수: 315
fold2 데이터 수: 318
fold3 데이터 수: 318
fold4 데이터 수: 318
fold5 데이터 수: 319


In [ ]:
import pandas as pd
import numpy as np

# Excel 파일 읽기
FNF_Excel = pd.read_excel('/workspace/FNF/labeling.xlsx', usecols=[0, 7, 9], engine='openpyxl', header=None)
FNF_Excel.columns = FNF_Excel.iloc[0]
FNF_Excel = FNF_Excel.iloc[1:]

# exclusion=2인 데이터 확인
exclusion2_data = FNF_Excel[FNF_Excel['exclusion'] == 2]
exclusion0_data = FNF_Excel[FNF_Excel['exclusion'] == 0]


# 1.0 제외 대상
exclude_1_0 = [51, 259, 598, 554, 1334]

# Internal Raw Dicom 제외 대상
exclude_internal = [98, 141, 161, 228, 501, 748, 1592, 1769]

# LAT Detection box 추가 데이터
add_lat = [808, 1175, 1363, 1587]

# Exclusion=0에서 각 데이터 존재 여부 확인
print("1.0 제외 대상 중 Exclusion=0에 있는 데이터:")
check_1_0 = FNF_Excel[
    (FNF_Excel['exclusion'] == 0) & 
    (FNF_Excel['serial No.'].isin(exclude_1_0))
]
print(check_1_0[['serial No.', 'exclusion', '정답label(by CT)']])

print("\nInternal Raw Dicom 제외 대상 중 Exclusion=0에 있는 데이터:")
check_internal = FNF_Excel[
    (FNF_Excel['exclusion'] == 0) & 
    (FNF_Excel['serial No.'].isin(exclude_internal))
]
print(check_internal[['serial No.', 'exclusion', '정답label(by CT)']])

print("\nLAT Detection box 추가 데이터 중 Exclusion=0에 있는 데이터:")
check_lat = FNF_Excel[
    (FNF_Excel['exclusion'] == 0) & 
    (FNF_Excel['serial No.'].isin(add_lat))
]
print(check_lat[['serial No.', 'exclusion', '정답label(by CT)']])
# 레이블 종류별로 분류
def classify_label(label):
    if pd.isna(label):
        return 'NaN'
    elif isinstance(label, (int, float)):
        return f'Type {int(label)}'
    elif '?' in str(label):
        return str(label)
    return str(label)

# 결과 출력
print("Exclusion=2 데이터 (31개):")
print(exclusion2_data[['serial No.', 'exclusion', '정답label(by CT)']].sort_values('serial No.'))

print("\n레이블 종류별 개수:")
label_counts = exclusion2_data['정답label(by CT)'].apply(classify_label).value_counts()
print(label_counts)

# 결과 출력
print("Exclusion=0 데이터 (1558개):")
print(exclusion0_data[['serial No.', 'exclusion', '정답label(by CT)']].sort_values('serial No.'))

print("\nExclusion=0 데이터 레이블 종류별 개수:")
label_counts = exclusion0_data['정답label(by CT)'].apply(classify_label).value_counts()
print(label_counts)


exclude_serials = [51, 259, 598, 554, 1334]  # 1.0에서 제외되어야 할 데이터
add_serials = [808, 1175, 1363, 1587]  # 1.1 & 2.0에서 추가된 데이터

# exclusion=0 데이터 중 제외 대상 확인
exclusion0_exclude = FNF_Excel[
    (FNF_Excel['exclusion'] == 0) & 
    (FNF_Excel['serial No.'].isin(exclude_serials))
]
print("Exclusion=0에서 제외되어야 할 데이터:")
print(exclusion0_exclude[['serial No.', 'exclusion', '정답label(by CT)']])

# exclusion=0 데이터 중 추가된 데이터 확인
exclusion0_add = FNF_Excel[
    (FNF_Excel['exclusion'] == 0) & 
    (FNF_Excel['serial No.'].isin(add_serials))
]
print("\nExclusion=0에 이미 포함된 추가 데이터:")
print(exclusion0_add[['serial No.', 'exclusion', '정답label(by CT)']])

# 제외/추가 데이터의 Type 확인
exclude_serials = [51, 259, 598, 554, 1334]
add_serials = [808, 1175, 1363, 1587]

print("제외 데이터의 Type:")
exclude_data = FNF_Excel[FNF_Excel['serial No.'].isin(exclude_serials)]
print(exclude_data[['serial No.', 'exclusion', '정답label(by CT)']])

print("\n추가 데이터의 Type:")
add_data = FNF_Excel[FNF_Excel['serial No.'].isin(add_serials)]
print(add_data[['serial No.', 'exclusion', '정답label(by CT)']])